In [1]:
import dataset

ds = dataset.Dataset()

Using TensorFlow backend.


In [2]:
import numpy as np
from tqdm import tqdm

def generate_training_data(num_samples=10000, input_shape=(32, 32, 3)):
    inputs = []
    targets = []
    
    for i in tqdm(range(num_samples)):
        np.random.seed(i)
        inp_seq = []
        for _ in range(input_shape[0]):
            inp_seq.append([i])
        inputs.append(inp_seq)
        targets.append(np.random.normal(size=input_shape))
    return np.array(inputs), np.array(targets)

In [3]:
inputs, targets = generate_training_data()

100%|██████████| 10000/10000 [00:04<00:00, 2314.78it/s]


In [4]:
inputs.shape

(10000, 32, 1)

In [5]:
from keras.layers import Conv2D, Dense, Input, LSTM, TimeDistributed, Reshape
from keras.models import Model

def create_random_lstm():
    inp = Input(shape = (32, 1))
    x = LSTM(64, return_sequences=True)(inp)
    x = LSTM(32*3, return_sequences=True)(x)
    out = Reshape((32, 32, 3))(x)
#     out = TimeDistributed(Dense(32))(x)
#     out = LSTM(32, return_sequences=True)(x)

    model = Model(inputs=inp, outputs=out)
    model.summary()

    return model



In [6]:
from keras.optimizers import SGD, Nadam
from clr_callback import CyclicLR

def fit(model, inputs=inputs, targets=targets):
    optim = Nadam()
    base_lr = 0.001
    max_lr = 0.006
    clr = CyclicLR(base_lr=base_lr, max_lr=max_lr,
                   step_size=2000., mode='triangular')

    model.compile(optimizer=optim,
                  loss="binary_crossentropy")
    
    model.fit(inputs, targets, batch_size=32, callbacks=[clr])

In [7]:
lstm = create_random_lstm()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 1)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 32, 96)            86400     
_________________________________________________________________
reshape_1 (Reshape)          (None, 32, 32, 3)         0         
Total params: 284,544
Trainable params: 284,544
Non-trainable params: 0
_________________________________________________________________


In [8]:
fit(lstm)

Epoch 1/1
10000/10000 [==============================] - 111s - loss: 0.0262   


In [14]:
test = []
np.random.seed(15000)
for i in range(32):
    test.append([i])
test = np.expand_dims(np.array(test), axis=0)

test.shape

(1, 32, 1)

In [15]:
lstm.predict(test)

array([[[[ -1.21873600e-04,   1.15705933e-03,  -2.98443629e-04],
         [ -3.73026051e-05,  -2.32145842e-03,  -3.89676279e-04],
         [ -4.41823417e-04,  -1.37211510e-03,  -3.41061270e-03],
         ..., 
         [ -3.21893650e-03,  -5.78174600e-04,  -1.44652207e-03],
         [  6.80531841e-04,  -1.12809939e-03,  -3.64669831e-04],
         [ -6.46311382e-04,   1.56376872e-03,  -3.89734702e-03]],

        [[  7.01516343e-04,   1.36453216e-03,  -2.04538764e-03],
         [ -1.23376935e-03,  -5.77161601e-03,  -2.92312732e-04],
         [  2.91653443e-04,  -2.70547229e-03,  -6.54486800e-03],
         ..., 
         [ -6.17278088e-03,  -4.49262065e-04,  -4.59703151e-03],
         [  7.96186796e-04,  -3.03367176e-03,  -2.62434012e-03],
         [ -1.39401457e-03,   5.81764383e-04,  -7.63486233e-03]],

        [[  3.19246086e-03,  -7.27943625e-05,  -6.80391118e-03],
         [ -4.64481581e-03,  -1.20641934e-02,   4.93440079e-04],
         [  2.64385366e-03,  -4.99774236e-03,  -1.014746